In [11]:
import itertools

class BayesianNode:
    def __init__(self, name, parents, cpt):
        self.name = name
        self.parents = parents
        self.cpt = cpt

    def probability(self, value, parent_values):
        parent_tuple = tuple(parent_values[parent] for parent in self.parents)
        prob = self.cpt[parent_tuple]
        return prob if value == 1 else 1 - prob

class BayesianNetwork:
    def __init__(self):
        self.nodes = {}

    def add_node(self, name, parents, cpt):
        self.nodes[name] = BayesianNode(name, parents, cpt)

    def get_node(self, name):
        return self.nodes[name]

    def inference(self, query_var, evidence):
        query_probs = {}

        for query_value in [0, 1]:
            extended_evidence = evidence.copy()
            extended_evidence[query_var] = query_value
            prob = self.enumerate_all(list(self.nodes.keys()), extended_evidence)
            query_probs[query_value] = prob

        # Normalize the probabilities
        total_prob = sum(query_probs.values())
        for key in query_probs:
            query_probs[key] /= total_prob

        return query_probs

    def enumerate_all(self, vars, evidence):
        if not vars:
            return 1.0

        # Ensure vars is a list
        vars = list(vars)

        first = vars[0]
        rest = vars[1:]
        node = self.get_node(first)

        if first in evidence:
            prob = node.probability(evidence[first], {k: evidence[k] for k in node.parents if k in evidence})
            return prob * self.enumerate_all(rest, evidence)
        else:
            total_prob = 0
            for value in [0, 1]:  # Assuming binary variables (0 or 1)
                new_evidence = evidence.copy()
                new_evidence[first] = value
                prob = node.probability(value, {k: new_evidence[k] for k in node.parents if k in new_evidence})
                total_prob += prob * self.enumerate_all(rest, new_evidence)
            return total_prob

def build_mystery_network():
    network = BayesianNetwork()

    # Adding nodes to the network
    network.add_node('Motive', [], {(): 0.6})  # Prior probability of a motive
    network.add_node('AlibiDan', [], {(): 0.1})  # 10% chance of having a questionable alibi
    network.add_node('AlibiPenny', [], {(): 0.2})  # 20% chance of having a questionable alibi
    network.add_node('Evidence', [], {(): 0.5})  # 50% chance of evidence linking to the crime scene
    network.add_node('Behavior', [], {(): 0.3})  # 30% chance of suspicious behavior
    network.add_node('WeaponFound', [], {(): 0.2})  # 20% chance of finding the weapon
    network.add_node('WitnessTestimony', [], {(): 0.4})  # 40% chance of a witness seeing a suspect

    # Conditional probabilities for guilt
    network.add_node('DetectiveDanGuilty', ['Motive', 'AlibiDan', 'Evidence'], {
        (0, 0, 0): 0.1, (0, 0, 1): 0.4, (0, 1, 0): 0.6, (0, 1, 1): 0.9,
        (1, 0, 0): 0.2, (1, 0, 1): 0.5, (1, 1, 0): 0.8, (1, 1, 1): 0.95
    })

    network.add_node('ProfessorPennyGuilty', ['Motive', 'AlibiPenny', 'Evidence'], {
        (0, 0, 0): 0.05, (0, 0, 1): 0.3, (0, 1, 0): 0.4, (0, 1, 1): 0.7,
        (1, 0, 0): 0.15, (1, 0, 1): 0.5, (1, 1, 0): 0.75, (1, 1, 1): 0.95
    })

    network.add_node('ChefCharlieGuilty', ['Motive', 'WitnessTestimony', 'WeaponFound', 'Behavior'], {
        (0, 0, 0, 0): 0.05, (0, 0, 0, 1): 0.2, (0, 0, 1, 0): 0.4, (0, 0, 1, 1): 0.6,
        (0, 1, 0, 0): 0.1, (0, 1, 0, 1): 0.5, (0, 1, 1, 0): 0.7, (0, 1, 1, 1): 0.9,
        (1, 0, 0, 0): 0.15, (1, 0, 0, 1): 0.4, (1, 0, 1, 0): 0.5, (1, 0, 1, 1): 0.7,
        (1, 1, 0, 0): 0.25, (1, 1, 0, 1): 0.6, (1, 1, 1, 0): 0.8, (1, 1, 1, 1): 0.95
    })

    return network

def main():
    print("Welcome to the Mystery Solver!")
    network = build_mystery_network()

    # Collect evidence from the user
    motive = int(input("Do you believe there was a motive (0 for No, 1 for Yes)? ").strip())
    alibi_dan = int(input("Does Detective Dan have a questionable alibi (0 for No, 1 for Yes)? ").strip())
    alibi_penny = int(input("Does Professor Penny have a questionable alibi (0 for No, 1 for Yes)? ").strip())
    evidence = int(input("Was there physical evidence linking one of them to the crime scene (0 for No, 1 for Yes)? ").strip())
    witness = int(input("Did a witness see one of the suspects near the crime scene (0 for No, 1 for Yes)? ").strip())
    weapon = int(input("Was there a weapon linked to one of the suspects (0 for No, 1 for Yes)? ").strip())
    behavior = int(input("Did any suspect display suspicious behavior (0 for No, 1 for Yes)? ").strip())

    # Create evidence dictionary
    evidence = {
        'Motive': motive,
        'AlibiDan': alibi_dan,
        'AlibiPenny': alibi_penny,
        'Evidence': evidence,
        'WitnessTestimony': witness,
        'WeaponFound': weapon,
        'Behavior': behavior
    }

    # Inference for Detective Dan
    dan_prob = network.inference('DetectiveDanGuilty', evidence)
    print(f"Probability of Detective Dan being guilty: {dan_prob[1]:.4f}")

    # Inference for Professor Penny
    penny_prob = network.inference('ProfessorPennyGuilty', evidence)
    print(f"Probability of Professor Penny being guilty: {penny_prob[1]:.4f}")

    # Inference for Chef Charlie
    charlie_prob = network.inference('ChefCharlieGuilty', evidence)
    print(f"Probability of Chef Charlie being guilty: {charlie_prob[1]:.4f}")

if __name__ == "__main__":
    main()


Welcome to the Mystery Solver!


Do you believe there was a motive (0 for No, 1 for Yes)?  0
Does Detective Dan have a questionable alibi (0 for No, 1 for Yes)?  0
Does Professor Penny have a questionable alibi (0 for No, 1 for Yes)?  0
Was there physical evidence linking one of them to the crime scene (0 for No, 1 for Yes)?  0
Did a witness see one of the suspects near the crime scene (0 for No, 1 for Yes)?  1
Was there a weapon linked to one of the suspects (0 for No, 1 for Yes)?  1
Did any suspect display suspicious behavior (0 for No, 1 for Yes)?  1


Probability of Detective Dan being guilty: 0.1000
Probability of Professor Penny being guilty: 0.0500
Probability of Chef Charlie being guilty: 0.9000
